In [74]:
import numpy as np

class SchemesPy: 
    soln = []
    
    def __init__(self, v_field, start, stop, h, init_conditions):
        self.vector_field = v_field
        self.ics = init_conditions
        self.a = start
        self.b = stop
        self.times = np.arange(self.a,self.b,h)  
        self.h = h
          
    def prep_preds(scheme):
        def wrapper(self):
            n = self.vector_field(self.ics, 
                                  self.times[0]).size
            x = np.zeros(((self.times).size,
                          n))
            x[0,:] = self.ics
            self.soln = scheme(self,x).flatten()
            return self.times, self.soln
        return wrapper
 
    @prep_preds
    def euler(self, x):
        for k, t in enumerate(self.times[:-1]):
            x[k+1,:] = x[k,:]+self.h*self.vector_field(x[k,:],t)
        return x
    
    @prep_preds
    def rk4(self, x):
        for k, t in enumerate(self.times[:-1]):
            f1 = self.vector_field(x[k,:],t)
            f2 = self.vector_field(x[k,:]+0.5*self.h*f1,t+0.5*self.h)
            f3 = self.vector_field(x[k,:]+0.5*self.h*f2,t+0.5*self.h)
            f4 = self.vector_field(x[k,:]+self.h*f3,t+self.h)
            x[k+1,:] = x[k,:]+self.h/6*(f1+2*f2+2*f3+f4)
        return x
    
    def __str__(self):
        return np.dstack((self.times, self.soln))
        

In [75]:
dy_dt = lambda y,t : 1 - t + 4*y
y_pred = SchemesPy(v_field=dy_dt, start=0, stop=2, h=0.01, init_conditions=1)

y_pred.rk4()
y_pred.__str__()

array([[[0.00000000e+00, 1.00000000e+00],
        [1.00000000e-02, 1.05096279e+00],
        [2.00000000e-02, 1.10390339e+00],
        [3.00000000e-02, 1.15890251e+00],
        [4.00000000e-02, 1.21604415e+00],
        [5.00000000e-02, 1.27541577e+00],
        [6.00000000e-02, 1.33710836e+00],
        [7.00000000e-02, 1.40121664e+00],
        [8.00000000e-02, 1.46783921e+00],
        [9.00000000e-02, 1.53707867e+00],
        [1.00000000e-01, 1.60904181e+00],
        [1.10000000e-01, 1.68383981e+00],
        [1.20000000e-01, 1.76158833e+00],
        [1.30000000e-01, 1.84240781e+00],
        [1.40000000e-01, 1.92642357e+00],
        [1.50000000e-01, 2.01376605e+00],
        [1.60000000e-01, 2.10457101e+00],
        [1.70000000e-01, 2.19897977e+00],
        [1.80000000e-01, 2.29713940e+00],
        [1.90000000e-01, 2.39920297e+00],
        [2.00000000e-01, 2.50532981e+00],
        [2.10000000e-01, 2.61568574e+00],
        [2.20000000e-01, 2.73044335e+00],
        [2.30000000e-01, 2.8497822

### Algorithm 4.1 (Spectral Deferred Correction).
##### Comment [Compute initial approximation]
For non-stiff/stiff problems, use the forward/backward Euler method to
compute an approximate solution φ i ≈ φ(s i ) at the nodes s 1 , . . . , s m on
the interval [a, b].
##### Comment [Compute successive corrections.]
    do j = 1, . . . , J
        1) Compute the approximate residual function 
            σ(φ [j−1] ) = .
        2a) For non-stiff problems, compute δ [j] = C exp (G, σ(φ [j−1] ))
        2b) For stiff problems, compute δ [j] = C imp (G, σ(φ [j−1] ))
        3) Update the approximate solution φ [j] = φ [j−1] + δ [j] .
    enddo"""

In [82]:
import scipy as sp

class SDC(SchemesPy):
    
    """ For non-stiff problems which are numerically stable. 
    Stiff problems are when certain numerical methods for solving 
        the equation are numerically unstable, unless the step size is 
        taken to be extremely small.
    """
    def sdc(self):
        psi = []
        # Initial approximation
        ts, f_euler = self.euler()
        psi.append(f_euler)
        
        m=10
        
        # Gauss Legenre nodes, s1 .. sm on [a,b]
        gl_nodes = [(self.b - self.a)/2 *x + (self.b+self.a)/2
                    for x in np.polynomial.legendre.leggauss(m)]
        
        c = lambda i,t : np.product([(t - ts[j])/(ts[i]-ts[j]) 
                                     for j in range(1,m+1) if i!=j])
        
        # Lagrange Interpolant
        L_m = lambda psi, t : np.sum([ c(i,t)*psi[i] 
                                      for i in range(1,m+1)])
        # Integration matrix
        S_m = lambda f : [sp.integrate(L_m(f,t), -1, t_i, args=(f))
                          for t_i in ts]
        
        # Successive corrections
        for j in range(1, len(f_euler)):
            # (j-1)th approximate solution psi[j-1]
            psi[j-1] = 
            
            
            # Residual = S^m _F(psi[j-1])- psi(j-1) + ^psi_a
            residual = S_m()
            
            # For non-stiff problems (C_exp)
            
            # Update approximate solution

In [83]:
y = SDC(v_field=dy_dt, start=0, stop=2, h=0.01, init_conditions=1).sdc()

[array([0.02609347, 0.13493663, 0.32059043, 0.56660461, 0.85112566,
       1.14887434, 1.43339539, 1.67940957, 1.86506337, 1.97390653]), array([1.06667134, 1.14945135, 1.21908636, 1.26926672, 1.29552422,
       1.29552422, 1.26926672, 1.21908636, 1.14945135, 1.06667134])]
